# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
# import dependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Experiment, Dataset
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.data.dataset_factory import TabularDatasetFactory
#from azureml.train.hyperdrive import uniform, choice
from azureml.core import ScriptRunConfig
from azureml.core import Environment
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

import os
import shutil


## Initialize Workspace and Create an Azure ML experiment

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

AZML
RG-AZML
eastus
a748a023-896b-4719-8f8e-3d1ba62d6e35


In [3]:
experiment_name = 'ideb-regression'

experiment=Experiment(ws, experiment_name)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
datastore = ws.get_default_datastore()
path = "data/data.csv"
try:
    ideb_dataset = Dataset.get_by_name(ws, name="ideb_dataset")
except:
    datastore.upload('data', target_path='data')
    # Create TabularDataset & register in workspace
    ideb_dataset = Dataset.Tabular.from_delimited_files([(datastore, path)])
    ideb_dataset = ideb_dataset.register(
        ws, name="ideb_dataset", create_new_version=True,
        description="Dataset for ideb prediction"
    )

## Creating Compute

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Jobrunning.................................................................................................................................................................................................................
Running


## Create an environment

In [52]:
%%writefile conda_dependencies.yml

dependencies:
  - python=3.6.2
  - scikit-learn
  - numpy
  - pip:
    - azureml-defaults

Overwriting conda_dependencies.yml


In [53]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [54]:

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(
                        evaluation_interval=2,
                        slack_factor=0.1
)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( 
    {
         '--max_depth': choice(range(1, 20)),
         '--min_samples_split': choice(2, 5, 10, 15, 100),
         '--min_samples_leaf': choice(range(0, 10))
    }
)

#TODO: Create your estimator and hyperdrive config
#estimator = SKLearn(
#                source_directory="./",
#                entry_script="train.py",
#                compute_target=cpu_cluster_name
#)

src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      compute_target = compute_target,
                      environment=sklearn_env)

hyperdrive_config = HyperDriveConfig(
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=20,
                    max_concurrent_runs=4,
                    hyperparameter_sampling=param_sampling,
                    policy=early_termination_policy,
                    #estimator=estimator,
                    run_config=src
)


##  Submit Run

In [55]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [56]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_98321714-3098-4c39-83fc-88bc580956e7
Web View: https://ml.azure.com/experiments/ideb-regression/runs/HD_98321714-3098-4c39-83fc-88bc580956e7?wsid=/subscriptions/a748a023-896b-4719-8f8e-3d1ba62d6e35/resourcegroups/RG-AZML/workspaces/AZML

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-21T02:26:28.539517][API][INFO]Experiment created<END>\n""<START>[2021-01-21T02:26:29.973661][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2021-01-21T02:26:29.793140][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-01-21T02:26:31.0979652Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_98321714-3098-4c39-83fc-88bc580956e7
Web View: https://ml.azure.com/experiments/ideb-regression/runs/HD_98321714-3098-4c39-83fc-88bc580956e7?wsid=/subscriptions/a748a023-896b-4719-8f8e-3d1ba62d6e35/r

{'runId': 'HD_98321714-3098-4c39-83fc-88bc580956e7',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-21T02:26:28.251288Z',
 'endTimeUtc': '2021-01-21T02:35:02.370703Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '7f4e5578-8db5-47c4-b555-4051ddbdd6c9',
  'score': '0.9979035203141273',
  'best_child_run_id': 'HD_98321714-3098-4c39-83fc-88bc580956e7_17',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://azml8724251163.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_98321714-3098-4c39-83fc-88bc5

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [57]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
print(hyperdrive_run.get_metrics())



Run(Experiment: ideb-regression,
Id: HD_98321714-3098-4c39-83fc-88bc580956e7_17,
Type: azureml.scriptrun,
Status: Completed)
{'HD_98321714-3098-4c39-83fc-88bc580956e7_0': {'max_depth:': 8, 'min_samples_split:': 5, 'min_samples_leaf:': 6, 'Accuracy': 0.9966351800180151}, 'HD_98321714-3098-4c39-83fc-88bc580956e7_1': {'max_depth:': 19, 'min_samples_split:': 10, 'min_samples_leaf:': 2, 'Accuracy': 0.9976211234064082}, 'HD_98321714-3098-4c39-83fc-88bc580956e7_10': {'max_depth:': 19, 'min_samples_split:': 10, 'min_samples_leaf:': 5, 'Accuracy': 0.9976051685221513}, 'HD_98321714-3098-4c39-83fc-88bc580956e7_11': {'max_depth:': 9, 'min_samples_split:': 15, 'min_samples_leaf:': 0}, 'HD_98321714-3098-4c39-83fc-88bc580956e7_12': {'max_depth:': 5, 'min_samples_split:': 2, 'min_samples_leaf:': 4, 'Accuracy': 0.9781551868632736}, 'HD_98321714-3098-4c39-83fc-88bc580956e7_13': {'max_depth:': 12, 'min_samples_split:': 100, 'min_samples_leaf:': 6, 'Accuracy': 0.9885910156991847}, 'HD_98321714-3098-4c39-8

In [58]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics=best_run.get_metrics()
best_run_files=best_run.get_file_names()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run ID',best_run.id)
print('\n Metrics: ', best_run_metrics)
print('\n Parameters: ', parameter_values,sep='\n')
print('\nAccuracy of Best run',best_run_metrics['Accuracy'],sep='\n')
print('\nBest run file names',best_run_files,sep='\n')

Best Run ID HD_98321714-3098-4c39-83fc-88bc580956e7_17

 Metrics:  {'max_depth:': 14, 'min_samples_split:': 2, 'min_samples_leaf:': 2, 'Accuracy': 0.9979035203141273}

 Parameters: 
['--max_depth', '14', '--min_samples_leaf', '2', '--min_samples_split', '2']

Accuracy of Best run
0.9979035203141273

Best run file names
['azureml-logs/55_azureml-execution-tvmps_959b442c5a0ffa1d4b1ab577b7180775caf83e201c43664cdc30520d8654fe8c_d.txt', 'azureml-logs/65_job_prep-tvmps_959b442c5a0ffa1d4b1ab577b7180775caf83e201c43664cdc30520d8654fe8c_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_959b442c5a0ffa1d4b1ab577b7180775caf83e201c43664cdc30520d8654fe8c_d.txt', 'logs/azureml/95_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_75d6ae7e-f2e1-4c28-ba8f-81d35a9cb788.jsonl', 'logs/azureml/dataprep/python_span_l_75d6ae7e-f2e1-4c28-ba8f-81d35a9cb788.jsonl', 'logs/azureml/job_pr

## Save the best model

In [59]:
#TODO: Save the best model
model = best_run.register_model(model_name='best-hyper-model', model_path='outputs/model.pkl')
print('Best model registered')
model


model.download(target_dir="outputs", exist_ok=True)
print('model downloaded')

Best model registered
model downloaded


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [80]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.conda_dependencies import CondaDependencies

#env = Environment.get(workspace=ws, name="AzureML-Tutorial")
env.python.conda_dependencies.add_pip_package("scikit-learn")
#env = Environment.from_conda_specification(name='automl',file_path='conda_env_v_1_0_0.yml')
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True, auth_enabled=True)
inference_config = InferenceConfig(entry_script='score.py', environment=env)
model = best_run.register_model(
                                model_name = "best_hyper_model", 
                                model_path = 'outputs/model.pkl',
                                description = "Best model trained with Hyperdrive")
print(best_run)

service = model.deploy(
    workspace=ws,
    name="service-ideb-prediction",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)
service.wait_for_deployment(show_output=True)
print(service.state)

Run(Experiment: ideb-regression,
Id: HD_98321714-3098-4c39-83fc-88bc580956e7_17,
Type: azureml.scriptrun,
Status: Completed)
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........................................................................................................................................................................................................................................................................................................................................................................
TimedOut


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: fb10b9a3-ff54-41f5-9276-5175a5143950
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice. \nPlease refer to https://aka.ms/debugimage#dockerlog for more information."
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: fb10b9a3-ff54-41f5-9276-5175a5143950
More information can be found using '.get_logs()'
Error:
{
  "code": "DeploymentTimedOut",
  "statusCode": 504,
  "message": "The deployment operation polling has TimedOut. The service creation is taking longer than our normal time. We are still trying to achieve the desired state for the web service. Please check the webservice state for the current webservice health. You can run print(service.state) from the python SDK to retrieve the current state of the webservice. \nPlease refer to https://aka.ms/debugimage#dockerlog for more information."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Transitioning\nOperation ID: fb10b9a3-ff54-41f5-9276-5175a5143950\nMore inform

In [ ]:
print(service.state)

In [82]:
service.get_logs()

ERROR:azureml.core.webservice.webservice:Received bad response from Model Management Service:
Response Code: 409
Headers: {'Date': 'Thu, 21 Jan 2021 03:44:46 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'x-ms-client-request-id': '0a2a89bcb3fd49958acd9504050e01b4', 'x-ms-client-session-id': '7eb16c81-7c21-4150-9c0b-a82f471be5e4', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '1.127', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"Conflict","statusCode":409,"message":"Conflict","details":[{"code":"ContainerGroupDeploymentNotReady","message":"The container group \'service-ideb-prediction-YoWqpxFbJEaOVZpBkYgx6A\' is not ready for the operation."}],"correlation":{"RequestId":"0a2a89bcb3fd49958acd9504050e01b4"}}'



WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 409
Headers: {'Date': 'Thu, 21 Jan 2021 03:44:46 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'x-ms-client-request-id': '0a2a89bcb3fd49958acd9504050e01b4', 'x-ms-client-session-id': '7eb16c81-7c21-4150-9c0b-a82f471be5e4', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '1.127', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"Conflict","statusCode":409,"message":"Conflict","details":[{"code":"ContainerGroupDeploymentNotReady","message":"The container group \'service-ideb-prediction-YoWqpxFbJEaOVZpBkYgx6A\' is not ready for the operation."}],"correlation":{"RequestId":"0a2a89bcb3fd49958acd9504050e01b4"}}'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad resp

## Test the web service deployed

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

In [ ]:
import requests
import json

# scoring endpoint
scoring_uri = service.scoring_uri


# If the service is authenticated, set the key or token
#key = '<your key or token>'

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
           "Cod_Escola": 4,
           "TAprov2019_Total": 99.9, 
           "TAprov2019_1_serie": 99.2, 
           "TAprov2019_2_serie": 59.1, 
           "TAprov2019_3_serie": 60.5, 
           "TAprov2019_4_serie": 70.5, 
           "Indicador_Rendimento": 0.99, 
           "SAEB2019_Matematica": 365.38, 
           "SAEB2019_Lingua Portuguesa": 351.54, 
           "SAEB2019_Nota Media Padronizada": 7.055853, 
           "IDEB_2019": 7.1
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

## Print the logs of the web service and delete the service

In [ ]:
service.get_logs()
service.delete()